In [ ]:
import ctypes  
import mysql.connector
from tkinter import *
from functools import partial
import threading
import pandas as pd
from threading import Timer,Thread,Event
import winsound


In [ ]:
def ConectaMySql():
    global con
    global cursorMySql
    
    try:
        con = mysql.connector.connect(host='192.168.2.132', database='otrs', port=3306, user='painel_otrs_apl',password='painel_otrs_apl_1234', buffered=True)
    except mysql.connector.Error as err:
        print("Sem VPN...: {}".format(err))
        input("Pressione Qualquer Tecla Pra Continuar")

        return
    
    if con.is_connected():
        db_info = con.get_server_info()
        cursorMySql = con.cursor()
#        print("Conectado ao servidor MySQL PRODUÇÃO versão ",db_info)

In [ ]:
def CheckTicketNovo():
    global Flag, Tickets, Resposta, Sql
    
    Sql = "SELECT id FROM otrs.ticket \
           where \
           CAST(create_time AS date) = current_date \
           AND queue_id = 22 \
           and (ticket_state_id = 12 \
           or ticket_state_id = 11)"
    
    ConectaMySql()   
    cursorMySql.execute(Sql)
    
    if Flag == 0:
        Tickets = [linha[0] for linha in cursorMySql]
        Flag = 1
    else:
        Resposta = [linha[0] if linha[0] in Tickets else 999999 for linha in cursorMySql]
        print(cursorMySql.rowcount)
        if 999999 in Resposta:
            cursorMySql.close()
            con.close()
            ConectaMySql()
            cursorMySql.execute(Sql)
            Tickets = [linha[0] for linha in cursorMySql]
            frequency = 2500  # Set Frequency To 2500 Hertz
            duration = 1200  # Set Duration To 1000 ms == 1 second
            winsound.Beep(frequency, duration)
#            ctypes.windll.user32.MessageBoxW(0,  "Abertura de Chamado No OTRS", "Chamado OTRS", 0)
    
    cursorMySql.close()
    con.close()           
#CheckTicketNovo()        

In [ ]:
class perpetualTimer():

    def __init__(self,t, hFunction):
        self.t=t
        self.hFunction = hFunction
        self.thread = Timer(self.t,self.handle_function)

    def handle_function(self):
        self.hFunction()
        self.thread = Timer(self.t,self.handle_function)
        self.thread.start()

    def start(self):
        self.thread.start()

    def cancel(self):
        self.thread.cancel()

In [ ]:
__name__ = "__main__"

def start(arg):
    def starting(args):
        if args is 1:
            lVar.set("Iniciando Checagem")
            lbStatus["fg"] = "white"
            lbStatus["bg"] = "blue"
        if args is 2:
            lVar.set("Checagem Iniciando  .")
            lbStatus["fg"] = "white"
            lbStatus["bg"] = "blue"
        if args is 3:
            lVar.set("Checagem Iniciando  . .")
            lbStatus["fg"] = "white"
            lbStatus["bg"] = "blue"
        if args is 4:
            lVar.set("Checagem Iniciando . . .")
            lbStatus["fg"] = "white"
            lbStatus["bg"] = "blue"
    def running():
        lbStatus["fg"] = "white"
        lVar.set('[+] Checando Novos Tickets')
        lbStatus["bg"] = "green"
    def not_original():
        lbStatus["fg"] = "white"
        lbStatus["bg"] = "blue"
        lVar.set("[!] Click Butão Start ")
    def err_running():
        lbStatus["fg"] = "red"
        lVar.set('[!] Já está Rodando ')
        lbStatus["bg"] = "black"
        threading.Timer(1, running).start()
    def err_notRunning():
        lbStatus["fg"] = "red"
        lbStatus["bg"] = "black"
        lVar.set("[!] Sem Rodar ")
        threading.Timer(1, not_original).start()
    def repeat():
        threading.Timer(0.5,starting,[1]).start()
        threading.Timer(1, starting,[2]).start()
        threading.Timer(1.5, starting,[3]).start()
        threading.Timer(2, starting,[4]).start()
    global thr, Flag
    if arg == btStart:
        if thr is None:
           #Start Listener
            ConectaMySql()
            thr = perpetualTimer(3, CheckTicketNovo)
            threading.Timer(0.1, repeat).start()
            threading.Timer(2, repeat).start()
            thr.start()
            threading.Timer(5, running).start()
        else:
            print("Já rodando")
            err_running()
    if arg == btStop:
        if thr is None:
            print('Nao está rodando')
            err_notRunning()
        else:
            print("Parando o Processamento")
            threading.Timer(1, not_original).start()
            thr.cancel()
            thr = None
#            cursorMySql.close()
#            con.close()

def runInBackground():
     app.withdraw()

thr = None
Flag = 0

#Controller
app = Tk()

#Estado
lVar = StringVar()
lbStatus = Label(app, textvariable= lVar , bg="blue", fg="white")
lbStatus.pack(side=BOTTOM, fill=X)
lVar.set("[!] Click Botão Start")

#Button Start
btStart = Button(app, text="Start")
btStart.pack(side = TOP, fill=X)
btStart["command"] = partial(start, btStart)
#Button Stop
btStop = Button(app, text="Stop")
btStop.pack(side=TOP, fill=X)
btStop["command"] = partial(start, btStop)
#Button Backg
btBackground = Button(app, text="Rodar in Background", command=runInBackground)
btBackground.pack(side=TOP, fill=X)

#Window Configs
app.geometry("300x150+800+400")
app.title("Bismarck K. Hegermann")
app.maxsize(width= 300, height= 150)
app.minsize(width= 300, height= 150)
app.mainloop()